In [0]:
import pandas as pd
from scipy.sparse import hstack
import torch
from sklearn import *
import sklearn
import re
from torchtext import data
import numpy as np
import torch.nn as nn

In [0]:
#read the data
available_data = pd.read_csv("quora_train_data.csv")
available_data = available_data

In [478]:
#split data into train and test
train_df, test_df = sklearn.model_selection.train_test_split(available_data, test_size=0.1, random_state=123)
train_df.shape, test_df.shape

((291088, 6), (32344, 6))

In [0]:
class Cleaning():
  def __init__(self, doc_cleaner_pattern=r"('\w+)|([^a-zA-Z0-9])"):
    #self.token_pattern = token_pattern 
    self.doc_cleaner_pattern = doc_cleaner_pattern
  def document_cleaner(self, 
                       lower=True):
    clean_doc_pattern = re.compile(self.doc_cleaner_pattern)
    if lower:
          return lambda doc: clean_doc_pattern.sub(" ", doc).lower()
    else:
          return lambda doc: clean_doc_pattern.sub(" ", doc)
  #def tokenizer(self):
    #token_pattern_aux = re.compile(self.token_pattern)
    #return lambda doc: token_pattern_aux.findall(doc)
  def fit(self, X): 
    data = []
    doc_cleaner = self.document_cleaner()
    #doc_tokenizer = self.tokenizer()
    for x in X:
      try: 
        x = doc_cleaner(x) 
        x = " ".join(x.split())
        #tokens = doc_tokenizer(x) 
        data.append(x)
      except: 
        data.append('nan')
    [data.strip() for data in data] 
    return data  

In [480]:
#cleaning train
cleaning = Cleaning()
train_data_1 = cleaning.fit(train_df['question1'])
train_df['question1'] = train_data_1
train_data_2 = cleaning.fit(train_df['question2'])
train_df['question2'] = train_data_2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
#drop not needed columns
train_df = train_df.drop(columns= ['id', 'qid1', 'qid2'])

In [0]:
train_df.to_csv('train_df.csv', index=False)

In [484]:
#cleaning test
cleaning = Cleaning()
test_data_1 = cleaning.fit(test_df['question1'])
test_df['question1'] = test_data_1
test_data_2 = cleaning.fit(test_df['question2'])
test_df['question2'] = test_data_2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
#drop columns not needed
test_df = test_df.drop(columns= ['id', 'qid1', 'qid2'])

In [486]:
test_df.head(10)

,question1,question2,is_duplicate
164818,what is it like to travel in afghanistan,what is it like for a foreigner to travel in a...,0
55016,do rappers really make as much money as they c...,can i make money from lyrics videos on youtube,0
270232,did mary give a natural vaginal birth to jesus...,how is the labour of mary and her birth giving...,0
204802,how can you determine the lewis structure for o2,how is the lewis structure for propane determined,0
286997,where does plex get poster art thumbnails for ...,who designed the ux and ui of plex media center,0
65642,is it possible to know if somebody else tumblr...,can anyone see a list of my secondary blogs i ...,0
290775,if you could have only money fame or love in y...,how does one define success is it the way one ...,0
132133,how do you say also in japanese is there more ...,how do you say ocean in japanese is there more...,0
151513,how good bad will it be for india if trump win...,if donald trump wins the presidential election...,1
286377,why can i see views on my instagram videos,how can i see who views my instagram,1


In [0]:
test_df.to_csv('test_df.csv', index=False)

In [0]:
#define fields
question1 = data.Field(tokenize='spacy')
question2 = data.Field(tokenize='spacy')
label = data.LabelField(dtype=torch.float)

fields = [('question1', question1), ('question2', question2), ('is_duplicate', label)]

train_data, test_data = data.TabularDataset.splits(path ='.', train= 'train_df.csv', test= 'test_df.csv', format = 'csv', fields = fields, skip_header = True)


In [489]:
#check one example of the train data
print(vars(train_data[0]))

{'question1': ['how', 'do', 'i', 'get', 'home', 'tutors'], 'question2': ['how', 'can', 'i', 'trust', 'a', 'home', 'tutor'], 'is_duplicate': '0'}


In [490]:
#check one example of the test data
print(vars(test_data[0]))

{'question1': ['what', 'is', 'it', 'like', 'to', 'travel', 'in', 'afghanistan'], 'question2': ['what', 'is', 'it', 'like', 'for', 'a', 'foreigner', 'to', 'travel', 'in', 'afghanistan'], 'is_duplicate': '0'}


In [0]:
#build vocabulary for training dataset using dictionary from torch text 
MAX_VOCAB_SIZE = 60000
question1.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
question2.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
label.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)

In [492]:
#length of vocabulary for training dataset (words + 1(unk) + 1(pad))
len(question1.vocab)

57999

In [493]:
#10 first words of the vocabuary for training dataset
question1.vocab.itos[:10]

['<unk>', '<pad>', 'the', 'what', 'is', 'how', 'i', 'a', 'to', 'in']

In [494]:
#the most common words of the vocabulary for training dataset 
question1.vocab.freqs.most_common(10)

[('the', 135988),
 ('what', 118202),
 ('is', 96812),
 ('how', 78726),
 ('i', 76428),
 ('a', 73513),
 ('to', 71317),
 ('in', 70669),
 ('of', 57751),
 ('do', 56921)]

In [0]:
#build vocabulary for test dataset using dictionary from torch text 
MAX_VOCAB_SIZE = 60000
question1.build_vocab(test_data, max_size = MAX_VOCAB_SIZE)
question2.build_vocab(test_data, max_size = MAX_VOCAB_SIZE)
label.build_vocab(test_data, max_size = MAX_VOCAB_SIZE)

In [496]:
#length of vocabulary for test dataset (words + 1(unk) + 1(pad))
len(question2.vocab)

20063

In [497]:
#10 first words of the vocabuary for test dataset
question2.vocab.itos[:10]

['<unk>', '<pad>', 'the', 'what', 'is', 'i', 'how', 'a', 'to', 'in']

In [498]:
#the most common words of the vocabulary for test dataset 
question2.vocab.freqs.most_common(10)

[('the', 14878),
 ('what', 12660),
 ('is', 10808),
 ('i', 9440),
 ('how', 9021),
 ('a', 8937),
 ('to', 8651),
 ('in', 7898),
 ('do', 6618),
 ('of', 6332)]

In [0]:
#create batches from train and test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    sort = False, #don't sort test/validation data
    batch_size=BATCH_SIZE,
    device=device)

In [501]:
#check it works
for batch in train_iterator:
    print(batch)
    break
batch.question2


[torchtext.data.batch.Batch of size 64]
	[.question1]:[torch.cuda.LongTensor of size 28x64 (GPU 0)]
	[.question2]:[torch.cuda.LongTensor of size 53x64 (GPU 0)]
	[.is_duplicate]:[torch.cuda.FloatTensor of size 64 (GPU 0)]


tensor([[  3,  26,   4,  ...,   4,  26,   3],
        [  4, 829,  19,  ...,  19,   4,   2],
        [ 35,   4, 115,  ..., 283,   2, 468],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')

In [502]:
#check it works
for batch in test_iterator:
    print(batch)
    break
batch.question1


[torchtext.data.batch.Batch of size 64]
	[.question1]:[torch.cuda.LongTensor of size 28x64 (GPU 0)]
	[.question2]:[torch.cuda.LongTensor of size 31x64 (GPU 0)]
	[.is_duplicate]:[torch.cuda.FloatTensor of size 64 (GPU 0)]


tensor([[   3,   11,   51,  ...,   39,   17,    3],
        [   4, 7594, 4509,  ...,   51,   11,   12],
        [  18,  117,  243,  ..., 9868,   40,    2],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

In [503]:
batch.question1

tensor([[   3,   11,   51,  ...,   39,   17,    3],
        [   4, 7594, 4509,  ...,   51,   11,   12],
        [  18,  117,  243,  ..., 9868,   40,    2],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

In [0]:
#model
class RNN(nn.Module):
    def __init__(self, input_dim1, input_dim2, embedding_dim=128, hidden_dim=128, output_dim=2, dropout=0.):
        super().__init__()
        self.embedding1 = nn.Embedding(input_dim1, embedding_dim)
        self.embedding2 = nn.Embedding(input_dim2, embedding_dim)
        self.rnn1 = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=2, dropout=dropout, bidirectional=True)
        self.rnn2 = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=2, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(4*hidden_dim, output_dim)
        self.hidden_dim = hidden_dim
        
    def forward(self, question1, question2):
        #question = [sent len, batch size]        
        embedded1 = self.embedding1(question1)    
        embedded2 = self.embedding2(question2)  
    
        #embedded = [sent len, batch size, emb dim]        
        output1, hidden1 = self.rnn1(embedded1)   
        output2, hidden2 = self.rnn2(embedded2)
        
        #output = [sent len, batch size, hid dim]
        output = torch.cat([output1[-1:,:,:], output2[-1:,:,:]])

        #output = [num rnn, batch size, hid dim * 2 (bidirectional)]
        output = output.permute(1,0,2)

        #output = [batch size, num rnn, hid dim * 2]
        output = output.reshape(-1, 4*self.hidden_dim)

        #output = [batch size, hid dim * 2 * 2 (num rnn)]
        y = self.fc(output).squeeze(1)    
        return y

In [0]:
#define gpu
device = torch.device('cuda')

In [513]:
#model to gpu
model = RNN(input_dim1=len(question1.vocab), input_dim2=len(question2.vocab))
model.to(device)
#model.load_state_dict(torch.load('model.pth'))

RNN(
  (embedding1): Embedding(21331, 128)
  (embedding2): Embedding(20063, 128)
  (rnn1): LSTM(128, 128, num_layers=2, bidirectional=True)
  (rnn2): LSTM(128, 128, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)

In [514]:
#check dimensions of output
output = model(batch.question1.to(device), batch.question2.to(device))
output.shape

torch.Size([64, 2])

In [516]:
epochs = 10
#criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1*10**(-3))
softmax = torch.nn.Softmax()

for epoch in range(1, epochs+1):
  loss_values = []
  accuracy_values = []
  for batch in train_iterator:
    output = model(batch.question1.to(device), batch.question2.to(device))
    loss = criterion(output, batch.is_duplicate.long().to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step() 
    loss_values.append(loss.item()) 
  print(epoch, np.mean(loss_values))
  for batch in test_iterator: 
    output = model(batch.question1.to(device), batch.question2.to(device))
    #output = torch.sigmoid(output)
    #output = output > 0.5
    output = torch.argmax(softmax(output), dim=1)
    accuracy = (output == batch.is_duplicate.to(device)).float().sum()
    accuracy_values.append(accuracy/BATCH_SIZE) 
  print(epoch, torch.mean(torch.stack(accuracy_values)).item())

1 0.5257336037267457


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


1 0.7487648725509644
2 0.4606495612819107
2 0.7693614363670349
3 0.401890283896389
3 0.772078812122345
4 0.3332802940984694
4 0.7819293737411499
5 0.2614405991337173
5 0.7746418118476868
6 0.19674538607733882
6 0.7730361223220825


KeyboardInterrupt: ignored

In [0]:
#torch.save(model.state_dict(),"model.pth")